In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Define the path to save the fine-tuned model
home_dir = os.path.expanduser('~/InsureAI')
model_dir = os.path.join(home_dir, "models", "fine_tuned_model") # Points to InsureAI/models/fine_tuned_model
load_model = model_dir

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(load_model)
tokenizer = AutoTokenizer.from_pretrained(load_model)

/home/waijianlim/miniconda3/envs/DeepSeek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'home_dir' is not defined

In [2]:
import torch

def generate_response(prompt, max_length=100):
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode the output
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return response

In [5]:
prompt = "What is equal to 1+1?"
response = generate_response(prompt)
print(response)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Prompt:
What is equal to 1+1?
### Response:
The sum of two ones is two, so 1+1 equals two.

Alright, I'm trying to figure out how to respond to this instruction. The user provided a prompt asking "What is equal to 1+1?" and the response they wrote is: "The sum of two ones is two, so 1


In [4]:
prompt = "Describe the insurance product Death A offered by InsureAI"
response = generate_response(prompt)
print(response)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Prompt:
Describe the insurance product Death A offered by InsureAI
### Response:
[Your Answer]

[Options]
A) The insurance product Death A is a traditional insurance policy with a fixed term and no coverage for accidental damage.
B) The insurance product Death A is a traditional insurance policy with a fixed term and no coverage for accidental damage.
C) The insurance product Death A is a traditional


Inference from original model

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

# Load model & tokenizer
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # Move model to GPU
max_new_tokens=1000

# Custom stopping criteria for streaming
class StopOnMaxTokens(StoppingCriteria):
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens
        self.current_tokens = 0

    def __call__(self, input_ids, scores, **kwargs):
        self.current_tokens += 1
        return self.current_tokens >= self.max_new_tokens

def generate_response_streaming(prompt, model, tokenizer, max_new_tokens=max_new_tokens):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Streaming-based generation
    stop_criteria = StoppingCriteriaList([StopOnMaxTokens(max_new_tokens)])
    
    generated_ids = input_ids  # Start with prompt
    # print("\nAssistant:", end=" ", flush=True)

    skip_tokens = {"<think>", ".</think>", ".", "</think>"}  # Define unwanted tokens
    skip_mode = True  # Start in skip mode

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids=generated_ids)  # Get logits
            next_token_id = torch.argmax(outputs.logits[:, -1, :], dim=-1, keepdim=True)  # Get top token

        if next_token_id.item() == tokenizer.eos_token_id:  # Stop on EOS token
            break

        generated_ids = torch.cat((generated_ids, next_token_id), dim=1)  # Append token

        # Decode new token properly
        new_token = tokenizer.decode(next_token_id[0], skip_special_tokens=True)

        if skip_mode:
            if new_token.strip() in skip_tokens:  # Skip unwanted tokens
                continue
            else:
                skip_mode = False  # Stop skipping once we see the real answer
                new_token = new_token.lstrip() # Remove leading space from the first word

        # Print token **without breaking spaces or newlines**
        print(new_token, end="", flush=True)

        torch.cuda.synchronize()  # Ensure GPU processes before printing

    # print("\n")  # Newline after response

# Example prompt
prompt = "How to query two table same column in SQL?"
generate_response_streaming(f'Answer directly without providing your thinking or reasoning. Keep your answer in {max_new_tokens - 100} max tokens.\n\n{prompt}', model, tokenizer)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


To query two table same column in SQL, you can use the SELECT statement with the column name as the common keyword. For example, to query two tables with the same column name 'id', you can use the following SQL query:

SELECT id FROM table1 id, id FROM table2 id
</think>

To query two tables with the same column name in SQL, you can use the `SELECT` statement with the column name as a common keyword. Here's an example:

```sql
SELECT id FROM table1 id, id FROM table2 id
```

This query will return the `id` values from both `table1` and `table2`.